Saleh Ahmad
i200605@nu.edu.pk

In [1]:
import numpy as np
from collections import defaultdict
import random

In [2]:
class Bigram_Backward_LM_Roman_Urdu():
    def __init__(self):
        self.ngram = 2

    def fit(self,TrainCorpusVocab): #TrainCorpus = all tokens in order of occurence
        Vocabs = defaultdict(int)
        UniqueVocabSize = defaultdict(int)
        VocabSize = defaultdict(int)
        for ngram in range(self.ngram,0,-1):
            IdxToken = 0
            ngrams = defaultdict(int)
            Len_Vocab = len(TrainCorpusVocab)
            while IdxToken < Len_Vocab-ngram+1:
                current_n_gram = TrainCorpusVocab[IdxToken]
                for idx in range(1,ngram):
                    current_n_gram += ' ' + TrainCorpusVocab[IdxToken+idx]
                
                ngrams[current_n_gram] += 1
                IdxToken += 1

            ngrams = {k:v for k,v in ngrams.items() if v > 0}
            Vocabs[ngram] = ngrams
            UniqueVocabSize[ngram] = len(ngrams)
            VocabSize[ngram] = sum(ngrams.values())
        
        self.Vocabs = Vocabs
        self.UniqueVocabSizes = UniqueVocabSize
        self.VocabSizes = VocabSize

    def __Get_Random_Length_Of_Verse(self):
        #generate random length b/w 5 and 9
        return random.randint(6,8)

    def __Get_Random_Ending_Token(self,e_tokens):
        e_tokens = np.unique(e_tokens)
        #generate random starting token from e_tokens which is a list of ending tokens
        return random.choice(e_tokens)
    
    def __Print_Vocab(self):
        print(self.Vocabs)
    

    def __Get_Next_Word_From_Candidate(self,Sequence):
        NGramToUse = self.ngram
        Candidates = [ngram for ngram in self.Vocabs[NGramToUse].keys() if ''.join(ngram.split()[-1]) == Sequence]
                
        if len(Candidates) == 0:
            #return any one from most used 10 words in self.Vocabs[NGramToUse-1]
            ToReturn = random.choice(list(self.Vocabs[NGramToUse-1].keys())[:10])
            return ToReturn
        else:
            Probability_Of_Candidates = np.zeros_like(Candidates)
            for idx,candidate in enumerate(Candidates):
                # Probability_Of_Candidates[idx] = (self.Vocabs[NGramToUse][candidate] + 1) / (self.Priors[NGramToUse-1][Sequence] + self.UniqueVocabSizes[self.ngram])
                Probability_Of_Candidates[idx] = (self.Vocabs[NGramToUse][candidate] + 1) / (self.VocabSizes[self.ngram] + self.UniqueVocabSizes[self.ngram])
            return Candidates[np.argmax(Probability_Of_Candidates)].split()[-2]

    def predict(self,EndingTokens): #StartingTokens = list of repetitive tokens, can extract unique and count
        '''
        Prediction format
        1 stanza = 14 verses (1 * 14)
        1 verse = 6 - 8 words
        '''        

        OrderedVerse = ''
        for idxVerse in range(0,14):
            EndingTokenOfThisVerse = self.__Get_Random_Ending_Token(EndingTokens) #Verse ends with a random ending token, length is 1 right now
            Verse = EndingTokenOfThisVerse
            Sequence = EndingTokenOfThisVerse

            for idxWord in range(1,self.__Get_Random_Length_Of_Verse()):
                PrevWord = self.__Get_Next_Word_From_Candidate(Sequence)
                Verse = PrevWord + ' ' + Verse
                Sequence = PrevWord
            
            OrderedVerse += Verse + '\n'
            if idxVerse != 0 and idxVerse % 2 == 1:
                OrderedVerse += '\n'
        return OrderedVerse 

    def Print_Details(self):
        print('All ngrams',self.Vocabs.keys())
        for ngram in self.Vocabs.keys():
            print('ngram:',ngram)
            print('ngrams:',list(self.Vocabs[ngram].keys())[0:10])

In [3]:
AllTokens = np.load('../Data/All_Tokens_Roman_Urdu.npy')
AllTokens = [token.lower() for token in AllTokens]
EndingTokens = np.load('../Data/Ending_Tokens_Roman_Urdu.npy')
EndingTokens = [token.lower() for token in EndingTokens]

In [4]:
ObjLM = Bigram_Backward_LM_Roman_Urdu()
ObjLM.fit(AllTokens)
# ObjLM.Print_Details()

In [5]:
Pred = ObjLM.predict(EndingTokens)
print(Pred)

dard ka jahan dil hi se darakhshan
tha maida lazzat dard ka nuzool kitaab

nah aaye nah aaye nah khmosh bathye
hain meri na-ash ko day tilsam

theen kurbateen kitni baham saghar o
sehan gulshan mein jurrat aazma lena

bacha ke ae kash فسون niaz mandi
bacha ke gum kya be panah tera ایاغ

ka jahan dil hi se aati daagh
apna bana hai naqsh qadam mera

ne apna bana hai shab hijran
aaye nah aaye nah jayoo daman nichor

lazzat dard ka jahan dil guzar
to yeh kis ne apna bana hai فغفوری


